In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Tensorflow datasets

Part of the vision of TensorFlow to make machine learning and deep learning easier to learn and easier to use, is the concept of having built-in data sets. You seen the little bit of a preview of the way back in the first course, when the fashion MNEs was available to you without you needing to download and split the data into training a test sets. Expanding on this, there's a library called TensorFlow Data Services or TFDS for short, and that contains many data sets and lots of different categories. [Here](https://www.tensorflow.org/datasets/catalog/overview)'s some examples.


And while we can see that there are many different data sets for different types, particularly image-based, there's also a few for text, and we'll be using the IMDB reviews dataset next. This dataset is ideal because it contains a large body of texts, 50,000 movie reviews which are categorized as positive or negative. It was authored by Andrew Mass et al at Stanford, and you can learn more about it at this link.

## IMDB reviews dataset

Please find the link to he IMDB reviews dataset here

You will find here 50,000 movie reviews which are classified as positive of negative. 

In [1]:
import tensorflow as tf
print(tf.__version__)

# !pip install -q tensorflow-datasets

2.3.1


In [2]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)


Shuffling and writing examples to /Users/az01661/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteNF7P45/imdb_reviews-train.tfrecord


Shuffling and writing examples to /Users/az01661/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteNF7P45/imdb_reviews-test.tfrecord


Shuffling and writing examples to /Users/az01661/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteNF7P45/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /Users/az01661/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [85]:
import numpy as np

train_data, test_data = imdb['train'], imdb['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

# str(s.tonumpy()) is needed in Python3 instead of just s.numpy()
for s,l in train_data:
  training_sentences.append(s.numpy().decode('utf8'))
  training_labels.append(l.numpy())
  
for s,l in test_data:
  testing_sentences.append(s.numpy().decode('utf8'))
  testing_labels.append(l.numpy())
  
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)


In [86]:
vocab_size = 1000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)


In [87]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[3]))

? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? this is the kind of film for a <OOV> <OOV> <OOV> when the rest of the world can go <OOV> with its own business as you <OOV> into a big <OOV> <OOV> and <OOV> for a couple of hours wonderful performances from <OOV> and <OOV> <OOV> as always <OOV> <OOV> the plot along there are no <OOV> to <OOV> no <OOV> <OOV> just a <OOV> and <OOV> <OOV> through new york life at its best a family film in every sense and one that <OOV> the <OOV> it <OOV>


## Embeddings

<img src="figures/embeddings.png" width="600px">

An embedding layer is similar to a dense layer. The two main advantages of Embedding over Dense layers are reduced input size and reduce computational complexity, which results in speeding up the training time. For more details, check this [aricle](https://medium.com/logivan/neural-network-embedding-and-dense-layers-whats-the-difference-fa177c6d0304) 



In [88]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 120, 16)           16000     
_________________________________________________________________
flatten_3 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 6)                 11526     
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 7         
Total params: 27,533
Trainable params: 27,533
Non-trainable params: 0
_________________________________________________________________


In [89]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 2s 2ms/step - loss: 0.5328 - accuracy: 0.7127 - val_loss: 0.3945 - val_accuracy: 0.8229
Epoch 2/10
782/782 [==============================] - 1s 2ms/step - loss: 0.3871 - accuracy: 0.8264 - val_loss: 0.3865 - val_accuracy: 0.8252
Epoch 3/10
782/782 [==============================] - 2s 2ms/step - loss: 0.3317 - accuracy: 0.8598 - val_loss: 0.4032 - val_accuracy: 0.8166
Epoch 4/10
782/782 [==============================] - 1s 2ms/step - loss: 0.2727 - accuracy: 0.8912 - val_loss: 0.4494 - val_accuracy: 0.8030
Epoch 5/10
782/782 [==============================] - 2s 2ms/step - loss: 0.2172 - accuracy: 0.9184 - val_loss: 0.5164 - val_accuracy: 0.7897
Epoch 6/10
782/782 [==============================] - 2s 2ms/step - loss: 0.1663 - accuracy: 0.9412 - val_loss: 0.6124 - val_accuracy: 0.7832
Epoch 7/10
782/782 [==============================] - 2s 2ms/step - loss: 0.1225 - accuracy: 0.9610 - val_loss: 0.6934 - val_accuracy: 0.7735
Epoch 

In [90]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(1000, 16)


In [91]:
word_index

{'<OOV>': 1,
 'the': 2,
 'and': 3,
 'a': 4,
 'of': 5,
 'to': 6,
 'is': 7,
 'br': 8,
 'in': 9,
 'it': 10,
 'i': 11,
 'this': 12,
 'that': 13,
 'was': 14,
 'as': 15,
 'for': 16,
 'with': 17,
 'movie': 18,
 'but': 19,
 'film': 20,
 'on': 21,
 'not': 22,
 'you': 23,
 'are': 24,
 'his': 25,
 'have': 26,
 'he': 27,
 'be': 28,
 'one': 29,
 'all': 30,
 'at': 31,
 'by': 32,
 'an': 33,
 'they': 34,
 'who': 35,
 'so': 36,
 'from': 37,
 'like': 38,
 'her': 39,
 'or': 40,
 'just': 41,
 'about': 42,
 "it's": 43,
 'out': 44,
 'if': 45,
 'has': 46,
 'some': 47,
 'there': 48,
 'what': 49,
 'good': 50,
 'more': 51,
 'when': 52,
 'very': 53,
 'up': 54,
 'no': 55,
 'time': 56,
 'she': 57,
 'even': 58,
 'my': 59,
 'would': 60,
 'which': 61,
 'only': 62,
 'story': 63,
 'really': 64,
 'see': 65,
 'their': 66,
 'had': 67,
 'can': 68,
 'were': 69,
 'me': 70,
 'well': 71,
 'than': 72,
 'we': 73,
 'much': 74,
 'been': 75,
 'bad': 76,
 'get': 77,
 'will': 78,
 'do': 79,
 'also': 80,
 'into': 81,
 'people': 82,
 '

In [92]:
word_num = 17

In [100]:
reverse_word_index[19]

'but'

In [101]:
weights[18]

array([-0.08712974, -0.02620778,  0.00133084, -0.0409667 , -0.0360341 ,
       -0.12397804,  0.01780314,  0.10849738, -0.00845553, -0.00848405,
       -0.02139743, -0.00197246, -0.04562975,  0.02843424,  0.02214294,
       -0.02834546], dtype=float32)

In [95]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [96]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences([sentence])
print(sequence)

[[11, 64, 102, 12, 7, 478, 1]]
